In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from qiskit_aer import Aer
from qiskit.compiler import transpile
from qiskit.quantum_info import Statevector, DensityMatrix, random_statevector, partial_trace

from hypersphere import Hypersphere

from PIL import Image

from scipy.special import sph_harm
from scipy.linalg import sqrtm
import pandas as pd
import pickle

import numpy as np
import secrets, time, uuid, hashlib, random

Image Size: 64 x 64
Image Size: 64 x 64
Image Size: 64 x 64
4096


In [2]:
image_path = "/Users/devaldeliwala/quantum_image_encryption/images/el_primo_square.jpg"
save_image_path = "/Users/devaldeliwala/quantum_image_encryption/images/"
name = "el_primo_encrypted"
depth = 10
n = 8
encrypted_name = "el_primo_square_encrypted"
verbose = False

im = Image.open(image_path, 'r')
image = im.convert("RGB")

width, height = image.size

In [3]:
wave_functions, magnitudes = Hypersphere(n = n, 
                                        image_path = image_path, 
                                        verbose = verbose
).generate_statevectors()

def generate_seed():

    entropy = secrets.token_bytes(16) + time.time_ns().to_bytes(8, 'big') + uuid.uuid4().bytes
    seed = int(hashlib.sha256(entropy).hexdigest(), 16) % 2**32

    return seed

seeds = []
circuits = []

for i in range(len(wave_functions)): 

    num_wave_functions = n - 1

    # generate a randomized circuit of a certain depth using 
    # our generated seed
    seed = generate_seed() 
    circuit = random_circuit(num_wave_functions, depth = depth, seed = seed)

    circuits.append(circuit)
    seeds.append(seed)

for i in wave_functions: 
    display(i.draw('latex'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [4]:
groups = []

for i in range(len(wave_functions)):
    group = []
    for j in range(n):
        group.append(wave_functions[(i + j) % len(wave_functions)])
    groups.append(group)


In [5]:
simulator = Aer.get_backend('statevector_simulator')
final_wave_functions = []

# cyclic permutation of a list
def run(wave_func, i):

    result_wf = []
    main_circuit = circuits[i]

    # create a circuit to first prepare the initial_wave_function 
    # from a circuit starting from |0>|0>|0>...
    initialization_circuit = QuantumCircuit(n - 1)
    initialization_circuit.initialize(wave_func.data, 
                                        [j for j in range(n - 1 )])

    # append the randomized circuit to the initialization_circuit
    full_circuit = initialization_circuit.compose(main_circuit)

    # transpile the circuit onto the simulator backend
    transpiled_circuit = transpile(full_circuit, simulator)
    result = simulator.run(transpiled_circuit).result()

    # output the resultant statevector
    final_wave_function = result.get_statevector()
    result_wf.append(final_wave_function)

    return result_wf

final_wave_functions = []

for i in range(len(groups)): 
    wave_functions = groups[i]
    ancillary_wave_functions = []

    for j in wave_functions: 
        ancillary_wave_functions.append(run(j, i))
    
    final_wave_functions.append(ancillary_wave_functions)



In [6]:
density_matrices = []

# converts the resultant pure states into their density matrices
for i in final_wave_functions: 
    coupled_density_matrices = [] 

    for j in range(len(i)): 
        coupled_density_matrices.append(DensityMatrix(i[j][0]))

    density_matrices.append(coupled_density_matrices)

In [7]:
# generating a new seed based on the original seeds generated 
# for the randomized circuits
combined_string = ''.join(map(str, seeds))
hash_object = hashlib.sha256(combined_string.encode())
hex_dig = hash_object.hexdigest()[:5]
seed = int(hex_dig, 16)

# generate a temporary randomized statevector whhich we will 
# use to calculate fidelities for each of the pure states
randomized_pure_state = random_statevector(dims = 2**(n - 1), seed = seed)
randomized_density_matrix = DensityMatrix(randomized_pure_state)

fidelities = []

def calculate_fidelitiy(rho, sigma): 

    sqrt_rho = sqrtm(rho)
    product = np.dot(sqrt_rho, np.dot(sigma, sqrt_rho))
    sqrt_product = sqrtm(product)
    trace = np.trace(sqrt_product)
    
    return np.real(trace)**2

# calculate all the fidelities and store them in a list
for coupled_states in density_matrices: 
    coupled_fidelities = []

    for matrix in coupled_states: 

        fidelity = calculate_fidelitiy(matrix, randomized_density_matrix)
        coupled_fidelities.append(fidelity)

    fidelities.append(coupled_fidelities)

weights = [] 

# using the calculated fidelities, generate self.n density matrix
# coefficients that together sum to one
for values in fidelities: 
    coupled_weights = []
    total = 0 

    for fidelity in values: 
        total += fidelity 

    for fidelity in values: 
        coupled_weights.append(fidelity / total)

    weights.append(coupled_weights)

mixed_density_matrices = []

# generate mixed_states using the weights for each of the pure states
# the mixed states are just generated via ∑ weight_n * |Ψ_n><Ψ_n| 
for coupled_states_idx in range(len(density_matrices)):
    mixed_density_matrix = np.zeros((2**(n-1), 2**(n-1)), dtype=np.complex128)

    for matrix_idx in range(len(density_matrices[coupled_states_idx])):
        current_matrix = density_matrices[coupled_states_idx][matrix_idx].data

        mixed_density_matrix += weights[coupled_states_idx][matrix_idx] * current_matrix


    mixed_density_matrices.append(DensityMatrix(mixed_density_matrix))


In [8]:
full_eigenvalues, full_eigenvectors = [], []

# perform spectral/eigendecomposition on the resultant mixed states
# to retrieve the eigenvalues and eigenvectors 
for matrix in mixed_density_matrices: 

    is_unit_trace = np.isclose(np.trace(matrix), 1.0)
    is_hermitian = np.allclose(matrix, np.matrix(matrix.data).getH())

    # implement checks to ensure the provided density matrix is valid
    if not is_unit_trace: 
        raise Exception("Tr(ρ) ≠ 1")
    if not is_hermitian: 
        raise Exception("ρ not Hermitian")

    eigenvalues, eigenvectors = np.linalg.eigh(matrix.data)
    full_eigenvalues.append([eigenvalues])
    full_eigenvectors.append([eigenvectors])

    reconstructed_matrix = sum(eigenvalues[i] * np.outer(eigenvectors[:, i], 
                                np.conj(eigenvectors[:, i])) for i in range(len(eigenvalues)))

    spectral_decomp_worked = np.allclose(matrix, reconstructed_matrix)

    if not spectral_decomp_worked: 
        raise Exception("spectral decomposition failed")

In [9]:
eigenvalues, eigenvectors = full_eigenvalues, full_eigenvectors
entropies = []

# calculating the von-neumann entropy of the mixed_states 
# using their eigenvalues
def calculate_entropy(eigenvalues):  

    eigenvalues = np.array(eigenvalues)
    non_zero_eigenvalues = eigenvalues[eigenvalues > 0]

    entropy = -np.sum(non_zero_eigenvalues * np.log(non_zero_eigenvalues))

    return entropy

for eigenlist in eigenvalues: 
    entropies.append(calculate_entropy(eigenlist))

In [10]:
sorted_indices = np.argsort(entropies)[::-1]

entropies = np.array(entropies)
eigenvalues = np.array(eigenvalues)
eigenvectors = np.array(eigenvectors)

eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[sorted_indices]

# extracting all the zero eigenvalues that represent impossible states
# to retrieve upon measurement
non_zero_eigenvalue_idxs = []
for eigenlist in eigenvalues: 
    non_zero_eigenvalue_idxs.append(np.where(eigenlist[0] > 1e-10)[0])

organized_eigenvectors = []
statevectors = []

# organize all the statevectors based on decreasing entropy
# the eigenvectors from the mixed states with maximum entropy 
# are placed in the lowest index
for idx in range(len(eigenvectors)):
    allowed_idxs = non_zero_eigenvalue_idxs[idx]

    organized_eigenvectors.append(eigenvectors[idx][0][allowed_idxs])

for coupled_statevectors in organized_eigenvectors: 
    for statevector in coupled_statevectors: 
        statevectors.append(Statevector(statevector))

#for i in statevectors: 
#    display(i.draw('latex'))

In [11]:
organized_eigenvecs = []

for i in range(len(eigenvalues)):
    group = [] 
    for j in range(len(eigenvalues[i][0])):
        group.append(eigenvalues[i][0][j])
    group_no0 = np.where(np.array(group) > 1e-10)
    minimum = np.where(min(eigenvalues[i][0][group_no0]))
    organized_eigenvecs.append(eigenvectors[i][0][group_no0[0][minimum]])

statevectors = []
for i in organized_eigenvecs: 
    statevectors.append(Statevector(i[0]))

/var/folders/38/q4k6mt054znfh_g97c8zyx1w0000gn/T/ipykernel_81610/3670572819.py:8: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(arr).nonzero()` if the old behavior was intended.
  minimum = np.where(min(eigenvalues[i][0][group_no0]))


In [19]:
n2_dimensional_unit_vectors = []

def statevector_to_unit_vector(statevector):
    real_parts = np.real(statevector)
    imag_parts = np.imag(statevector)
    combined = np.concatenate([real_parts, imag_parts])
    unit_vector = combined / np.linalg.norm(combined)
    return unit_vector

for i in statevectors: 
    n2_dimensional_unit_vectors.append(statevector_to_unit_vector(i))

print(n2_dimensional_unit_vectors[0].shape)

(256,)


In [11]:
def generate_seed():

    entropy = secrets.token_bytes(16) + time.time_ns().to_bytes(8, 'big') + uuid.uuid4().bytes
    seed = int(hashlib.sha256(entropy).hexdigest(), 16) % 2**32

    return seed

seeds = []
circuits = []

for i in range(len(wave_functions)): 

    num_wave_functions = num_qubits

    # generate a randomized circuit of a certain depth using 
    # our generated seed
    seed = generate_seed() 
    circuit = random_circuit(num_wave_functions, depth = 10, seed = seed)

    circuits.append(circuit)
    seeds.append(seed)

print(len(circuits))


NameError: name 'num_qubits' is not defined

In [ ]:
full_eigenvalues, full_eigenvectors = [], []

# perform spectral/eigendecomposition on the resultant mixed states
# to retrieve the eigenvalues and eigenvectors 
for matrix in density_matrices: 

    is_unit_trace = np.isclose(np.trace(matrix), 1.0)
    is_hermitian = np.allclose(matrix, np.matrix(matrix.data).getH())

    # implement checks to ensure the provided density matrix is valid
    if not is_unit_trace: 
        raise Exception("Tr(ρ) ≠ 1")
    if not is_hermitian: 
        raise Exception("ρ not Hermitian")

    eigenvalues, eigenvectors = np.linalg.eigh(matrix.data)
    full_eigenvalues.append([eigenvalues])
    full_eigenvectors.append([eigenvectors])

    reconstructed_matrix = sum(eigenvalues[i] * np.outer(eigenvectors[:, i], 
                                np.conj(eigenvectors[:, i])) for i in range(len(eigenvalues)))

    spectral_decomp_worked = np.allclose(matrix, reconstructed_matrix)

    if not spectral_decomp_worked: 
        raise Exception("spectral decomposition failed")

In [14]:
simulator = Aer.get_backend('statevector_simulator')
final_wave_functions = []

# cyclic permutation of a list
def run(wave_func, i):

    result_wf = []
    main_circuit = circuits[i]

    # create a circuit to first prepare the initial_wave_function 
    # from a circuit starting from |0>|0>|0>...
    initialization_circuit = QuantumCircuit(num_qubits)
    initialization_circuit.initialize(wave_func.data, 
                                        [j for j in range(num_qubits)])

    # append the randomized circuit to the initialization_circuit
    full_circuit = initialization_circuit.compose(main_circuit)

    # transpile the circuit onto the simulator backend
    transpiled_circuit = transpile(full_circuit, simulator)
    result = simulator.run(transpiled_circuit).result()

    # output the resultant statevector
    final_wave_function = result.get_statevector()
    result_wf.append(final_wave_function)

    return result_wf

final_wave_functions = []

for i in range(len(groups)): 
    wave_functions = groups[i]
    ancillary_wave_functions = []

    for j in wave_functions: 
        ancillary_wave_functions.append(run(j, i))
    
    final_wave_functions.append(ancillary_wave_functions)

final_wave_functions


[[[Statevector([-0.2031913 -0.509245j  ,  0.15479192-0.23623407j,
                -0.1718296 -0.47896884j,  0.01617492-0.1739043j ,
                 0.00524971+0.07578171j, -0.17499784+0.26604457j,
                 0.03149869+0.26574283j,  0.19963066+0.33397104j],
               dims=(2, 2, 2))],
  [Statevector([ 0.12179267+0.36800835j, -0.15901442+0.13347707j,
                -0.19707108-0.59463402j,  0.41359044-0.26810301j,
                 0.00990733+0.10836269j, -0.00104896-0.22608329j,
                -0.32614895+0.01384454j,  0.02443559+0.03397776j],
               dims=(2, 2, 2))],
  [Statevector([-0.04119479-0.0966321j , -0.09469587+0.39316364j,
                 0.06602715-0.38630188j, -0.09271752+0.48348393j,
                 0.20007403+0.16391145j, -0.11518172+0.2801025j ,
                -0.0784278 +0.0791038j , -0.12982487-0.49152278j],
               dims=(2, 2, 2))]],
 [[Statevector([ 0.15790457-0.05918039j, -0.06562085-0.48008275j,
                -0.05866963-0.10941381j

In [19]:
density_matrices = []

# converts the resultant pure states into their density matrices
for i in final_wave_functions: 
	coupled_density_matrices = [] 

	for j in range(len(i)): 
		coupled_density_matrices.append(DensityMatrix(i[j][0]))

	density_matrices.append(coupled_density_matrices)

for i in density_matrices: 
	display(i[0].draw('latex'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [26]:
# generating a new seed based on the original seeds generated 
# for the randomized circuits
combined_string = ''.join(map(str, seeds))
hash_object = hashlib.sha256(combined_string.encode())
hex_dig = hash_object.hexdigest()[:5]
seed = int(hex_dig, 16)

# generate a temporary randomized statevector whhich we will 
# use to calculate fidelities for each of the pure states
randomized_pure_state = random_statevector(dims = 2**num_qubits, seed = seed)
randomized_density_matrix = DensityMatrix(randomized_pure_state)

fidelities = []

def calculate_fidelitiy(rho, sigma): 

    sqrt_rho = sqrtm(rho)
    product = np.dot(sqrt_rho, np.dot(sigma, sqrt_rho))
    sqrt_product = sqrtm(product)
    trace = np.trace(sqrt_product)
    
    return np.real(trace)**2

# calculate all the fidelities and store them in a list
for coupled_states in density_matrices: 
    coupled_fidelities = []

    for matrix in coupled_states: 

        fidelity = calculate_fidelitiy(matrix, randomized_density_matrix)
        coupled_fidelities.append(fidelity)

    fidelities.append(coupled_fidelities)

fidelities

[[0.4360783793535708, 0.10021947255248093, 0.03528560592296333],
 [0.1529358650574516, 0.019946312419199325, 0.14879799285992024],
 [0.2099369243482714, 0.1510649824957417, 0.07101475543711573],
 [0.16624883547057392, 0.2788196190754988, 0.07181584015280396],
 [0.10279634614624643, 0.00014320901595617555, 0.03730981240197038],
 [0.01133668050825036, 0.18812563115378841, 0.36431994469903395],
 [0.15598012551756235, 0.012976470237065262, 0.12883536265349155],
 [0.18425831518146074, 0.39028608001611176, 0.15288419294890948],
 [0.013519826534545197, 0.35181896170881, 0.12653235416103928],
 [0.019023106075192153, 0.1812076440157567, 0.16734467303412215],
 [0.05928877820035931, 0.017023157166327435, 0.03441148868515575],
 [0.1416581504910472, 0.0205176863704947, 0.004575291429624841],
 [0.12229631937601723, 0.36942682210014194, 0.15280109797585434],
 [0.09897457861231425, 0.25389551972247915, 0.3867340161050599],
 [0.05938148940991792, 0.03638820561203585, 0.07395147201335876],
 [0.081808453

In [31]:
weights = [] 

# using the calculated fidelities, generate num_qubits density matrix
# coefficients that together sum to one
for values in fidelities: 
    coupled_weights = []
    total = 0 

    for fidelity in values: 
        total += fidelity 

    for fidelity in values: 
        coupled_weights.append(fidelity / total)

    weights.append(coupled_weights)

mixed_density_matrices = []

# generate mixed_states using the weights for each of the pure states
# the mixed states are just generated via ∑ weight_n * |Ψ_n><Ψ_n| 
for coupled_states_idx in range(len(density_matrices)):
    mixed_density_matrix = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)

    for matrix_idx in range(len(density_matrices[coupled_states_idx])):
        current_matrix = density_matrices[coupled_states_idx][matrix_idx].data

        mixed_density_matrix += weights[coupled_states_idx][matrix_idx] * current_matrix


    mixed_density_matrices.append(DensityMatrix(mixed_density_matrix))

for i in mixed_density_matrices: 
    display(i.draw('latex'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [39]:
full_eigenvalues, full_eigenvectors = [], []

# perform spectral/eigendecomposition on the resultant mixed states
# to retrieve the eigenvalues and eigenvectors 
for matrix in mixed_density_matrices: 

    is_unit_trace = np.isclose(np.trace(matrix), 1.0)
    is_hermitian = np.allclose(matrix, np.matrix(matrix.data).getH())

    # implement checks to ensure the provided density matrix is valid
    if not is_unit_trace: 
        raise Exception("Tr(ρ) ≠ 1")
    if not is_hermitian: 
        raise Exception("ρ not Hermitian")

    eigenvalues, eigenvectors = np.linalg.eigh(matrix.data)
    full_eigenvalues.append([eigenvalues])
    full_eigenvectors.append([eigenvectors])

    reconstructed_matrix = sum(eigenvalues[i] * np.outer(eigenvectors[:, i], 
                                np.conj(eigenvectors[:, i])) for i in range(len(eigenvalues)))

    spectral_decomp_worked = np.allclose(matrix, reconstructed_matrix)

    if not spectral_decomp_worked: 
        raise Exception("spectral decomposition failed")
    
print(full_eigenvalues[0])

[array([-2.29720968e-17, -1.38642345e-17,  1.67047408e-18,  1.70600926e-17,
        3.06906612e-17,  5.89384422e-02,  1.70323811e-01,  7.70737747e-01])]


In [34]:
print(full_eigenvectors)

[[array([[ 0.33032713+0.j        ,  0.16735412+0.j        ,
         0.12461545+0.j        , -0.21774183+0.j        ,
         0.53635014+0.j        , -0.1315286 +0.j        ,
        -0.4583355 +0.j        , -0.53375406+0.j        ],
       [ 0.39234916-0.05598368j, -0.35680365-0.39414882j,
         0.01602516-0.11491782j, -0.20635822+0.14722826j,
        -0.37519767-0.15333299j,  0.43355234-0.02059234j,
        -0.12629518-0.19618861j, -0.15654304-0.2256566j ],
       [ 0.08472765+0.04801429j, -0.2521322 +0.07833208j,
        -0.0884718 +0.17649974j,  0.26010321+0.12124003j,
         0.03062425-0.23519748j, -0.27360926+0.15021424j,
         0.53585114-0.30332362j, -0.51532098+0.03313043j],
       [ 0.26512761+0.04796527j,  0.24616352+0.31914035j,
         0.36056897+0.35225059j,  0.10141487+0.01219046j,
        -0.16542059+0.10004647j,  0.42950695+0.10394061j,
         0.22606718+0.42093491j, -0.18211552-0.07952252j],
       [-0.00415018-0.18319241j, -0.15259905-0.41357236j,
        

In [45]:
eigenvalues = full_eigenvalues
eigenvectors = full_eigenvectors

entropies = []

# calculating the von-neumann entropy of the mixed_states 
# using their eigenvalues
def calculate_entropy(eigenvalues):  

    eigenvalues = np.array(eigenvalues)
    non_zero_eigenvalues = eigenvalues[eigenvalues > 0]

    entropy = -np.sum(non_zero_eigenvalues * np.log(non_zero_eigenvalues))

    return entropy

for eigenlist in eigenvalues: 
    entropies.append(calculate_entropy(eigenlist))

print(entropies)

[0.6690580686965003, 0.8195736975717711, 0.872884018126579, 0.8725028573652807, 0.49005835553310617, 0.712791573799693, 0.707753081758423, 0.891736002410779, 0.6700174614550539, 0.7997733092723225, 0.8374649449559108, 0.4574719115092033, 0.8840256873570914, 0.908428350087302, 0.9631150796418813, 0.44171921038830547, 0.6975956214728871, 0.6462179222909694, 0.8607479915392237, 0.9474704963478325, 0.8468857529952672, 0.832771957545549, 0.7157346898127448, 0.7242874134726712, 0.4191371005921089, 0.6032040635469154, 0.8862293163324408, 0.8465132301132849, 0.7452212133120674, 1.0117567995063854, 0.9014887858667631, 0.6423984894243583]


In [48]:
sorted_indices = np.argsort(entropies)[::-1]

entropies = np.array(entropies)
eigenvalues = np.array(eigenvalues)
eigenvectors = np.array(eigenvectors)

eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[sorted_indices]

# extracting all the zero eigenvalues that represent impossible states
# to retrieve upon measurement
non_zero_eigenvalue_idxs = []
for eigenlist in eigenvalues: 
    non_zero_eigenvalue_idxs.append(np.where(eigenlist[0] > 1e-10)[0])

organized_eigenvectors = []
statevectors = []

# organize all the statevectors based on decreasing entropy
# the eigenvectors from the mixed states with maximum entropy 
# are placed in the lowest index
for idx in range(len(eigenvectors)):
    allowed_idxs = non_zero_eigenvalue_idxs[idx]

    organized_eigenvectors.append(eigenvectors[idx][0][allowed_idxs])

for coupled_statevectors in organized_eigenvectors: 
    for statevector in coupled_statevectors: 
        statevectors.append(Statevector(statevector))

for i in statevectors: 
    display(i.draw('latex'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>